In [ ]:
#ms = 20220410T0130  
#dt = datetime.fromtimestamp(ms/1000.0)
#dt

In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from statistics import mean

api_key = "APCAF6OLCGOVGNUS"
url_interest = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=monthlytopics&apikey={api_key}'
url_cpi = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey={api_key}'
url_coffee = f'https://www.alphavantage.co/query?function=COFFEE&interval=monthly&apikey={api_key}'

r1 = requests.get(url_interest)
r2 = requests.get(url_cpi)
r3 = requests.get(url_coffee)

data1 = r1.json()
data2 = r2.json()
data3 = r3.json()


print(data1)
print(data2)
print(data3)

In [ ]:

data1_table = pd.DataFrame(data1)
data2_table = pd.DataFrame(data2)
data3_table = pd.DataFrame(data3)

In [ ]:
values = [item["value"] for item in data1["data"]]
dates = [item["date"] for item in data1["data"]]
print(dates)
print(values)

In [ ]:
data1_table["us_interest_rate_%"] = values
data1_table["dates"] = dates

In [ ]:
del data1_table["data"]
del data1_table["name"]
del data1_table["interval"]
del data1_table["unit"]

In [ ]:
data1_table

In [ ]:
values2 = [item["value"] for item in data2["data"]]
dates2 = [item["date"] for item in data2["data"]]
print(dates2)
print(values2)

In [ ]:
data2_table["cpi"] = values2
data2_table["dates"] = dates2

In [ ]:
del data2_table["data"]
del data2_table["name"]
del data2_table["interval"]
del data2_table["unit"]
data2_table

In [ ]:
data3_table

In [ ]:
values3 = [item["value"] for item in data3["data"]]
dates3 = [item["date"] for item in data3["data"]]
print(dates3)
print(values3)

In [ ]:
data3_table

In [ ]:
data3_table["coffee"] = values3
data3_table["dates"] = dates3

In [ ]:
url_k = "./btc.csv"
df_kaggle = pd.read_csv(url_k)
print (df_kaggle)


In [ ]:
data3_table = pd.DataFrame(df_kaggle)

In [ ]:
data3_table